#### EĞİTİLMESİ GEREK HEPSİNİ ÇALITIRINCA EĞİTİLİYOR

In [ ]:
import librosa
import pandas as pd
import torch
from torch import nn
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

train_path = "Datasets/Filter_Datasets/train.csv"
test_path = "Datasets/Filter_Datasets/test.csv"

In [ ]:
def read_and_sample(csv_path, fraction):
    data = pd .read_csv(csv_path)
    sample_data = data.sample(frac=fraction, random_state=42)
    return sample_data

In [ ]:
fraction = 1

train_data = read_and_sample(train_path,fraction)
test_data = read_and_sample(test_path,fraction)

In [ ]:
train_data.head()

In [ ]:
target_sr = 48000

def data_clean(audio,sr):
    audio = librosa.resample(audio,orig_sr=sr,target_sr=target_sr)
    audio = librosa.util.normalize(audio)
    audio_feature = librosa.feature.melspectrogram(y=audio, sr=sr)
    audio_feature = librosa.power_to_db(S=audio_feature)
    return audio_feature

In [ ]:
def data_specktrogram(data):
    x_list, y_list = [], []

    for index, row in data.iterrows():
        audio, sample_rate = librosa.load(row["combined_path"])
        tensor_x = torch.from_numpy(data_clean(audio,sample_rate)).T.to(device)
        audio, sample_rate = librosa.load(row["clean_path"])
        tensor_y = torch.from_numpy(data_clean(audio,sample_rate)).T.to(device)
        x_list.append(tensor_x.unsqueeze(0))
        y_list.append(tensor_y.unsqueeze(0))
    return x_list, y_list

In [ ]:
train_x_list , train_y_list = data_specktrogram(train_data)

In [ ]:
test_x_list , test_y_list = data_specktrogram(test_data)

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, layer_size, output_size):
        super(LSTM, self).__init__()
        self.layer_size = layer_size
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size, hidden_size, layer_size, batch_first=True).to(device)
        self.fc = nn.Linear(hidden_size, output_size).to(device)
    def forward(self, x):
        h0 = torch.zeros(self.layer_size, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.layer_size, x.size(0), self.hidden_size).to(device)
        
        output, (hx, cx) = self.lstm(x, (h0, c0))
        output = self.fc(output)
        return output

In [ ]:
input_size = train_x_list[0].size(-1)
output_size = train_y_list[0].size(-1)
hidden_size = 256
layer_size = 5

model = LSTM(input_size, hidden_size, layer_size, output_size).to(device)

In [ ]:
epochs = 10000
lr = 0.001
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
iteration_history = list()
loss_history = list()
count = 0
show_loss = 1

for epoch in range(epochs):
    for audio, labels in zip(train_x_list, train_y_list):
        audio, labels = audio.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(audio)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        count += 1
        if count % show_loss == 0:
            iteration_history.append(count)
            loss_history.append(loss.item())
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch, epochs, loss.item()))
            torch.save(model,"Checkpoints")

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(iteration_history, loss_history, label='Loss')
plt.title('Training Loss')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
correct = 0
total = 0
model.eval()

for audio, labels in zip(test_x_list, test_y_list):
    audio, labels = audio.to(device), labels.to(device)
    outputs = model(audio)
    
    is_close = torch.isclose(outputs, labels)
    correct += is_close.sum().item()
    total += labels.numel()
accuracy = 100 * correct / total
print(accuracy)